# Initializing 

Setting Directory Paths for Training, Validation and Testing Data and Defining the Number of Classes as 4 - Grassy, marshy, Rocky, Sandy

In [ ]:
train_dir = r"D:\Terrain\Split\train"
valid_dir = r"D:\Terrain\Split\val"
test_dir = r"D:\Terrain\Split\test"
num_classes = 4

# Data Augmentation

Data Augmentation using the ImageDataGenerator module from TensorFlow's Keras API. Data augmentation is a common practice in deep learning, especially for image classification tasks, as it helps improve model generalization by artificially increasing the diversity of the training dataset.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

# Data Generation

Creation of data generators to load and preprocess image data from the specified directories for training, testing, and validation.  The generators apply appropriate data augmentation or preprocessing techniques as needed, depending on the purpose of each dataset.

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Loading MobileNetV2

Load the MobileNetV2 pre-trained model with Weights trained on ImageNet Dataset. We specify that we want to exclude the top (classification) layer and set the input shape to (224, 224, 3) to match the expected input size.

In [ ]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

# Building the Custom Classification Model

Adding Layers on Top of MobileNetV2 Base Model to Build a Custom Classification Model on top of the MobileNetV2 Model.

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compiling

Compiling the Custom Classification Model Using Adam Optimizer

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Training

Training the Custom Classification Model 

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Creating a Pandas Dataframe

Creating a Pandas Dataframe from Training History and Printing it.

In [ ]:
import pandas as pd
history_df = pd.DataFrame(history.history)
print(history_df)

# Testing the Model

Making Predictions Using the Trained Model on the Test Date  and Mapping the Predicted and True Class Indices to their Corresponding Class Names.

In [ ]:
preds = model.predict(test_generator)
y_pred = np.argmax(preds, axis=1)
for k, v in test_generator.class_indices.items():
    print(v, ":", k)
pred_classes = {
    0 : "Grassy",
    1 : "Marshy",
    2 : "Rocky",
    3 : "Sandy",
}
y_pred = list(map(lambda x: pred_classes[x], y_pred))
y_test = list(map(lambda x: pred_classes[x], test_generator.classes))

# Visualizing and Displaying Model Architecture

Creating a visual representation of the architecture of a deep learning model and displaying it within a Jupyter Notebook. The visual representation helps in comprehending the model's structure and can be essential for model development, understanding, and communication.

In [ ]:
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

img = plt.imread('model_plot.png')
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.show()

# Overview of the Model

Prove a training summary for the Model

In [ ]:
model.summary()

# Evaluating Model Performance and Generating Classification Report

Evaluating the performance of a deep learning model and generating a classification report for the model's predictions. It also calculates and prints the final accuracy of the model on the validation data.


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report


final_accuracy = history_df['accuracy'].iloc[-1]
print("Final Accuracy:", final_accuracy)


print(classification_report(y_pred, y_test))


# Saving the Trained Model

Saving the trained deep learning model to a file with a filename that includes a timestamp and the final accuracy achieved by the model on the validation data.

In [ ]:
import datetime as dt
from tensorflow.keras.models import load_model


date_time_format = '%Y_%m_%d__%H_%M_%S'


current_date_time_dt = dt.datetime.now()

current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

model_filename = f'terrain__{current_date_time_string}___Accuracy_{final_accuracy:.4f}.h5'

model.save(model_filename)
